<a href="https://colab.research.google.com/github/iceman67/-Python/blob/master/lab_missing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 수집된 공공데이터 ASOS 에 결측치 처리

In [ ]:
import pandas as pd
import requests
import json


In [ ]:
serviceKey = 'y80jnESQZu1%2B%2BKrpWpkGrnZ96%2FhiBicuIH%2F3SeO0u10CK9rglO3nqmwetj8%2BRHj%2F1NWUUis4aeGnUMk1CFUYRQ%3D%3D'
numOfRows = 31
startDt = 20210801
endDt = 20210831
stnId =108 # 지역
URL=f"http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey={serviceKey}&pageNo=1&numOfRows={numOfRows}&dataType=JSON&dataCd=ASOS&dateCd=DAY&startDt={startDt}&endDt={endDt}&stnIds={stnId}"


In [ ]:
def set_url(key, numOfRows, startDt, endDt, stnId):
    base = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"
    url =f"{base}?serviceKey={serviceKey}&pageNo=1&numOfRows={numOfRows}&dataType=JSON&dataCd=ASOS&dateCd=DAY&startDt={startDt}&endDt={endDt}&stnIds={stnId}"

    return url


In [ ]:
URL = set_url(serviceKey, numOfRows, startDt, endDt, stnId)
URL

* 주어진 URL 의 JSON 자료를 데이터프레임으로 구성함

In [ ]:
def gen_df(URL):
    result = requests.get(URL)
    js = json.loads(result.content)
    data = pd.DataFrame(js['response']['body']['items']['item'])
    return data
df = gen_df(URL)

* 자료값 확인을 위해 컬럼 번호와 자료형 (Dtype) 을 확인함

In [ ]:
df.info()

In [ ]:
df.head()

* 결측치를 확인함
> 결측치를 갖는 컬럼을 찾아봄 (예: mi10MaxRnHrmt)

In [ ]:
df.isnull().sum()

In [ ]:
# n99Rn 에 값을 확인함
df.iloc[3, 59:60]

* n99n 컬럼을 숫자로 변환한 후 결측치를 확인함

In [ ]:
df['n99Rn'] = pd.to_numeric(df['n99Rn'])
df

In [ ]:
df.isna().any()

In [ ]:
df['n99Rn'].head() 

In [ ]:
df.iloc[3, 59:60]

* 결측치 및  결측치 백분율로 확인

In [ ]:
df.isnull().sum()

In [ ]:
df.isnull().mean() 

* 주어진 데이터프레임의 값을 확인한 후 자료에 값이 없는 경우 숫자로 변환하여 결측치를 확인하시오.
> ddMesHrmt, sumFogDur

In [ ]:
df['sumFogDur'].head()

In [ ]:
df['sumFogDur'] = pd.to_numeric(df['sumFogDur'])
df['sumFogDur']

* 결측치 처리후의 문제점을 찾으시오

In [ ]:
df = df.dropna()
df

* 결측치를 대치 (중간값, 최빈값)

In [ ]:
df['sumFogDur'].median()

In [ ]:
df['sumFogDur_A'] = df['sumFogDur'].fillna(df['sumFogDur'].median())

In [ ]:
most_freq = df['sumFogDur'].value_counts()

In [ ]:
df['sumFogDur'] = df['sumFogDur'].fillna(most_freq)

* 결측치를 갖는 컬럼을 제거한다

In [ ]:
df.dropna(axis=1)

* 결측치 제거

> 결측값이 많은 컬럼은 제외함 (기준 50%)

In [ ]:
df = df.dropna(thresh=int(len(df)*0.5), axis =1)

In [ ]:
df.isnull().sum()

#### 참고문헌 

* https://wikidocs.net/16582